## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,77.0,92,59,1.99,broken clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,46.4,87,40,3.33,scattered clouds
2,2,Mataura,NZ,-46.1927,168.8643,63.0,68,32,7.00,scattered clouds
3,3,Las Vegas,US,36.1750,-115.1372,87.8,6,1,10.36,clear sky
4,4,Ilulissat,GL,69.2167,-51.1000,12.2,67,75,4.61,broken clouds


In [1]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,77.00,92,59,1.99,broken clouds
3,3,Las Vegas,US,36.1750,-115.1372,87.80,6,1,10.36,clear sky
10,10,Mahebourg,MU,-20.4081,57.7000,77.00,88,75,12.66,light rain
14,14,San Lorenzo,PY,-25.3333,-57.5333,78.01,83,0,4.61,clear sky
16,16,Avarua,CK,-21.2078,-159.7750,84.20,65,75,9.22,broken clouds
17,17,Port Lincoln,AU,-34.7333,135.8667,75.88,32,100,11.63,overcast clouds
18,18,Cabo San Lucas,MX,22.8909,-109.9124,82.40,36,40,10.36,scattered clouds
20,20,Dancalan,PH,9.9821,122.7386,80.62,66,100,1.97,overcast clouds
21,21,Rikitea,PF,-23.1203,-134.9692,77.02,61,0,8.46,clear sky
23,23,Denpasar,ID,-8.6500,115.2167,80.60,89,20,4.61,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 480
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              174 non-null    int64  
 1   City                 174 non-null    object 
 2   Country              174 non-null    object 
 3   Lat                  174 non-null    float64
 4   Lng                  174 non-null    float64
 5   Max Temp             174 non-null    float64
 6   Humidity             174 non-null    int64  
 7   Cloudiness           174 non-null    int64  
 8   Wind Speed           174 non-null    float64
 9   Current Description  174 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 15.0+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,
3,Las Vegas,US,87.80,clear sky,36.1750,-115.1372,
10,Mahebourg,MU,77.00,light rain,-20.4081,57.7000,
14,San Lorenzo,PY,78.01,clear sky,-25.3333,-57.5333,
16,Avarua,CK,84.20,broken clouds,-21.2078,-159.7750,
17,Port Lincoln,AU,75.88,overcast clouds,-34.7333,135.8667,
18,Cabo San Lucas,MX,82.40,scattered clouds,22.8909,-109.9124,
20,Dancalan,PH,80.62,overcast clouds,9.9821,122.7386,
21,Rikitea,PF,77.02,clear sky,-23.1203,-134.9692,
23,Denpasar,ID,80.60,few clouds,-8.6500,115.2167,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


    
    
  
   
    
   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Las Vegas,US,87.80,clear sky,36.1750,-115.1372,Plaza Hotel & Casino
10,Mahebourg,MU,77.00,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
14,San Lorenzo,PY,78.01,clear sky,-25.3333,-57.5333,Hotel del Sur
16,Avarua,CK,84.20,broken clouds,-21.2078,-159.7750,Paradise Inn


In [17]:
# 8a. Create the output File (CSV)
output_file = "../Vacation_Search/WeatherPy_Database.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))